<a href="https://colab.research.google.com/github/lauraosede/Machine-Learning-for-Insurance-Predicting-Claimants/blob/main/Regression_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Import Library


In [ ]:
#imports
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.svm import SVR
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder


## Exploratory Data Analysis

#### Load Data

In [ ]:
#loading the travel data sets
comp_data_path = '/content/drive/MyDrive/Design and application of a machine learning system for a practical problem/machine learning assessment/clients data/CE802_P3_Data.csv'
comp_data = pd.read_csv(comp_data_path)

In [ ]:
#Viewing summary of the data fields 
comp_data .describe()

,F1,F2,F3,F5,F6,F7,F8,F9,F10,F11,F13,F14,F15,F16,Target
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
mean,8.083320,8.109187,-185.816067,-224.532120,525.890600,63.602773,117.038520,3.947333,3.942667,6.490307,-15458.938340,-10.131380,-35.690000,1486.017107,922.082533
std,5.787817,5.812853,59.715228,87.920436,126.724179,774.154296,296.468097,1.836702,1.810036,6.080052,4443.035539,9.072751,12.549426,602.841367,1092.331874
min,0.080000,0.080000,-396.260000,-493.590000,-317.010000,0.000000,-947.190000,0.000000,0.000000,-13.640000,-35812.450000,-37.200000,-84.450000,-566.080000,0.000000
25%,3.890000,3.815000,-227.550000,-283.470000,465.892500,0.130000,-87.967500,3.000000,3.000000,2.580000,-17695.130000,-16.500000,-41.625000,1084.320000,0.000000
50%,6.830000,6.740000,-185.910000,-226.950000,525.705000,1.010000,119.865000,4.000000,4.000000,6.700000,-15459.995000,-10.140000,-35.745000,1478.850000,429.570000
75%,10.985000,10.945000,-145.720000,-166.440000,589.110000,6.805000,319.667500,5.000000,5.000000,10.450000,-13414.520000,-3.780000,-30.015000,1904.165000,1530.205000
max,46.780000,45.140000,51.620000,82.830000,1097.400000,27474.530000,1111.150000,11.000000,10.000000,25.900000,4302.780000,18.150000,47.160000,3587.780000,3960.010000


In [ ]:
comp_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   F1      1500 non-null   float64
 1   F2      1500 non-null   float64
 2   F3      1500 non-null   float64
 3   F4      1500 non-null   object 
 4   F5      1500 non-null   float64
 5   F6      1500 non-null   float64
 6   F7      1500 non-null   float64
 7   F8      1500 non-null   float64
 8   F9      1500 non-null   int64  
 9   F10     1500 non-null   int64  
 10  F11     1500 non-null   float64
 11  F12     1500 non-null   object 
 12  F13     1500 non-null   float64
 13  F14     1500 non-null   float64
 14  F15     1500 non-null   float64
 15  F16     1500 non-null   float64
 16  Target  1500 non-null   float64
dtypes: float64(13), int64(2), object(2)
memory usage: 199.3+ KB


In [ ]:
#Viweing Show first few rows in our dataframe
comp_data.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,Target
0,16.56,12.42,-236.06,Rest,-98.88,529.56,4.54,379.54,1,1,7.30,High,-15085.87,-12.93,-39.42,1734.58,3616.82
1,11.72,12.46,-190.06,Rest,-59.22,493.11,0.05,402.78,5,3,-1.28,Very low,-15782.44,-8.55,-35.61,1672.70,3342.88
2,4.34,2.74,-201.20,UK,-228.48,563.79,1.22,147.35,4,4,8.28,Low,-10526.01,-9.66,-29.10,1462.86,0.00
3,12.76,2.58,-282.26,UK,-173.28,536.94,0.25,113.49,4,3,6.26,Low,-8327.14,-19.23,-34.59,809.46,1742.65
4,11.10,9.82,-242.86,USA,-193.14,617.52,9.15,343.64,8,6,-6.88,Very low,-14434.13,-9.45,-46.14,1435.90,373.56


In [ ]:
#analyzing data in column
#get unique calues
print(comp_data[['F12']])
print(comp_data.F12.unique())
print(comp_data.F4.unique())

            F12
0          High
1      Very low
2           Low
3           Low
4      Very low
...         ...
1495  Very high
1496     Medium
1497     Medium
1498  Very high
1499   Very low

[1500 rows x 1 columns]
['High' 'Very low' 'Low' 'Very high' 'Medium']
['Rest' 'UK' 'USA' 'Europe']


#### convert non-numeric data into numeric data

In [ ]:
#converting F12 into float
comp_data['F12_encoded'] = comp_data['F12'].map( {'Very low': 0.0 ,'Low': 1.0, 'Medium': 2.0, 'Very high': 3.0, 'High': 4.0})
comp_data[['F12', 'F12_encoded']]

,F12,F12_encoded
0,High,4.0
1,Very low,0.0
2,Low,1.0
3,Low,1.0
4,Very low,0.0
...,...,...
1495,Very high,3.0
1496,Medium,2.0
1497,Medium,2.0
1498,Very high,3.0


In [ ]:
#spliting values in column F4 into other columns
#using one-hot encoding in F4
f4_encoded = pd.get_dummies(comp_data['F4'], prefix='F4')
f4_encoded


,F4_Europe,F4_Rest,F4_UK,F4_USA
0,0,1,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,1,0
4,0,0,0,1
...,...,...,...,...
1495,0,1,0,0
1496,0,0,1,0
1497,1,0,0,0
1498,0,0,0,1


In [ ]:
#adding newly converted data back into data frame
comp_data = pd.concat([f4_encoded,comp_data], axis = 1 )
comp_data

,F4_Europe,F4_Rest,F4_UK,F4_USA,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,Target,F12_encoded
0,0,1,0,0,16.56,12.42,-236.06,Rest,-98.88,529.56,4.54,379.54,1,1,7.30,High,-15085.87,-12.93,-39.42,1734.58,3616.82,4.0
1,0,1,0,0,11.72,12.46,-190.06,Rest,-59.22,493.11,0.05,402.78,5,3,-1.28,Very low,-15782.44,-8.55,-35.61,1672.70,3342.88,0.0
2,0,0,1,0,4.34,2.74,-201.20,UK,-228.48,563.79,1.22,147.35,4,4,8.28,Low,-10526.01,-9.66,-29.10,1462.86,0.00,1.0
3,0,0,1,0,12.76,2.58,-282.26,UK,-173.28,536.94,0.25,113.49,4,3,6.26,Low,-8327.14,-19.23,-34.59,809.46,1742.65,1.0
4,0,0,0,1,11.10,9.82,-242.86,USA,-193.14,617.52,9.15,343.64,8,6,-6.88,Very low,-14434.13,-9.45,-46.14,1435.90,373.56,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0,1,0,0,5.20,4.70,-218.72,Rest,-224.04,539.91,8.23,-232.93,2,3,12.32,Very high,-16977.67,5.01,-29.67,1420.16,1515.56,3.0
1496,0,0,1,0,12.10,2.98,-144.42,UK,-76.05,736.65,0.48,90.41,3,6,7.40,Medium,-13927.89,0.66,-36.06,1321.82,1528.48,2.0
1497,1,0,0,0,4.12,4.36,-160.30,Europe,-230.43,439.86,0.69,-224.44,3,3,7.02,Medium,-18373.26,-7.68,-39.42,1905.86,323.67,2.0
1498,0,0,0,1,1.58,13.50,-159.60,USA,-233.85,572.58,0.29,-126.79,1,5,1.68,Very high,-22863.72,-3.12,-54.36,2424.42,109.19,3.0


In [ ]:
comp_data = comp_data.drop(["F4", "F12"], axis = 1)


In [ ]:
comp_data

,F4_Europe,F4_Rest,F4_UK,F4_USA,F1,F2,F3,F5,F6,F7,F8,F9,F10,F11,F13,F14,F15,F16,Target,F12_encoded
0,0,1,0,0,16.56,12.42,-236.06,-98.88,529.56,4.54,379.54,1,1,7.30,-15085.87,-12.93,-39.42,1734.58,3616.82,4.0
1,0,1,0,0,11.72,12.46,-190.06,-59.22,493.11,0.05,402.78,5,3,-1.28,-15782.44,-8.55,-35.61,1672.70,3342.88,0.0
2,0,0,1,0,4.34,2.74,-201.20,-228.48,563.79,1.22,147.35,4,4,8.28,-10526.01,-9.66,-29.10,1462.86,0.00,1.0
3,0,0,1,0,12.76,2.58,-282.26,-173.28,536.94,0.25,113.49,4,3,6.26,-8327.14,-19.23,-34.59,809.46,1742.65,1.0
4,0,0,0,1,11.10,9.82,-242.86,-193.14,617.52,9.15,343.64,8,6,-6.88,-14434.13,-9.45,-46.14,1435.90,373.56,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0,1,0,0,5.20,4.70,-218.72,-224.04,539.91,8.23,-232.93,2,3,12.32,-16977.67,5.01,-29.67,1420.16,1515.56,3.0
1496,0,0,1,0,12.10,2.98,-144.42,-76.05,736.65,0.48,90.41,3,6,7.40,-13927.89,0.66,-36.06,1321.82,1528.48,2.0
1497,1,0,0,0,4.12,4.36,-160.30,-230.43,439.86,0.69,-224.44,3,3,7.02,-18373.26,-7.68,-39.42,1905.86,323.67,2.0
1498,0,0,0,1,1.58,13.50,-159.60,-233.85,572.58,0.29,-126.79,1,5,1.68,-22863.72,-3.12,-54.36,2424.42,109.19,3.0


In [ ]:
comp_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   F4_Europe    1500 non-null   uint8  
 1   F4_Rest      1500 non-null   uint8  
 2   F4_UK        1500 non-null   uint8  
 3   F4_USA       1500 non-null   uint8  
 4   F1           1500 non-null   float64
 5   F2           1500 non-null   float64
 6   F3           1500 non-null   float64
 7   F5           1500 non-null   float64
 8   F6           1500 non-null   float64
 9   F7           1500 non-null   float64
 10  F8           1500 non-null   float64
 11  F9           1500 non-null   int64  
 12  F10          1500 non-null   int64  
 13  F11          1500 non-null   float64
 14  F13          1500 non-null   float64
 15  F14          1500 non-null   float64
 16  F15          1500 non-null   float64
 17  F16          1500 non-null   float64
 18  Target       1500 non-null   float64
 19  F12_en

## Model evaluation

#### Train test split

In [ ]:
#selecting the prediction target
y = comp_data.Target

#choosing the prediction features
X = comp_data.drop('Target', axis = 1)

In [ ]:
#split  data into training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state = 0)


In [ ]:
feature_scaler = StandardScaler()
X_train = feature_scaler.fit_transform(X_train)
X_test = feature_scaler.transform(X_test)

#### Building our model


###### Building a linear regressor

In [ ]:
#defining the model
linear_reg_model = LinearRegression() 

#fitting the model to the training data
linear_reg_model.fit(X_train, y_train) 

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
#making predictions based on the test data
y_pred = linear_reg_model.predict(X_test) 

In [ ]:
#evaluate model accuracy using MSE
mean_square_error_comp = mean_squared_error(y_pred, y_test)
RMSE_num = np.sqrt(metrics.mean_squared_error(y_pred, y_test))
print('The Mean Square Error(MSE)  is: ',mean_square_error_comp)
print('R square is :', RMSE_num)

The Mean Square Error(MSE)  is:  275475.41020776116
R square is : 524.8575141957683


###### Building a random forest regressor

In [ ]:
#defining the model
ran_forest_model = RandomForestRegressor() 

#fitting the model to the training data
ran_forest_model.fit(X_train, y_train) 


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
#making predictions based on the test data
y_pred = ran_forest_model.predict(X_test) 

In [ ]:
#evaluate model accuracy using MSE
mean_square_error_comp = mean_squared_error(y_pred, y_test)
RMSE_num = np.sqrt(metrics.mean_squared_error(y_pred, y_test))
print('The Mean Square Error(MSE)  is: ',mean_square_error_comp)
print('R square is :', RMSE_num)

The Mean Square Error(MSE)  is:  340672.16293879447
R square is : 583.6712798646122


###### Building a support vector regressor

In [ ]:
#defining the model
support_vector_reg = SVR(C=1.0, epsilon=0.2)

#fitting the model to the training data
support_vector_reg.fit(X_train, y_train) 

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.2, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
#making predictions on test data 
y_pred = support_vector_reg.predict(X_test)


In [ ]:
#evaluate model accuracy using MSE
mean_square_error_comp = mean_squared_error(y_pred, y_test)
RMSE_num = np.sqrt(metrics.mean_squared_error(y_pred, y_test))
print('The Mean Square Error(MSE)  is: ',mean_square_error_comp)
print('R square is :', RMSE_num)

The Mean Square Error(MSE)  is:  1440014.967201649
R square is : 1200.0062363178156


#### Selecting best model

In [ ]:
#using cross validation
regressor =  linear_reg_model

pipeline = Pipeline(steps=[('scaler', sc),
                             ('regressor', regressor)])

param_grid = {'regressor__fit_intercept': [True,False]}

best_model = GridSearchCV(pipeline, param_grid, scoring = 'neg_mean_squared_error')
best_model.fit(X, y)

result = best_model.best_score_
print(result)

-266874.87915366364


In [ ]:
pred= best_model.predict(X_test)


### Predicition on hold out test set

In [ ]:
#load test data
data_path = '/content/drive/MyDrive/Design and application of a machine learning system for a practical problem/machine learning assessment/clients data/CE802_P3_Test.csv'
test_data = pd.read_csv(data_path)


In [ ]:
test_data.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,Target
0,28.70,0.82,-115.06,USA,-280.95,581.10,0.06,214.35,2,2,-0.88,Low,-14794.01,-15.09,-35.61,1578.68,NaN
1,5.40,6.90,-170.24,Rest,-228.00,330.54,0.31,-164.98,1,9,0.28,Medium,-11632.88,0.72,-29.73,1647.02,NaN
2,21.96,4.40,-219.12,Rest,-59.67,662.28,4.98,281.54,6,5,5.82,Low,-12631.29,-23.61,-32.73,1449.14,NaN
3,10.52,1.02,-184.14,UK,-211.23,506.55,0.57,-311.96,7,5,1.92,High,-26664.88,1.80,-22.92,911.98,NaN
4,2.00,0.58,-190.56,Rest,-284.82,649.41,0.05,291.47,8,3,-0.02,Low,-9901.41,-13.47,-48.81,2251.04,NaN


In [ ]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   F1      1500 non-null   float64
 1   F2      1500 non-null   float64
 2   F3      1500 non-null   float64
 3   F4      1500 non-null   object 
 4   F5      1500 non-null   float64
 5   F6      1500 non-null   float64
 6   F7      1500 non-null   float64
 7   F8      1500 non-null   float64
 8   F9      1500 non-null   int64  
 9   F10     1500 non-null   int64  
 10  F11     1500 non-null   float64
 11  F12     1500 non-null   object 
 12  F13     1500 non-null   float64
 13  F14     1500 non-null   float64
 14  F15     1500 non-null   float64
 15  F16     1500 non-null   float64
 16  Target  0 non-null      float64
dtypes: float64(13), int64(2), object(2)
memory usage: 199.3+ KB


#### convert non-numeric data into numeric data

In [ ]:
test_data.drop('Target', axis=1, inplace=True)

#### convert non-numeric data into numeric data

In [ ]:
#converting F12 into float
test_data['F12_encoded'] = test_data['F12'].map( {'Very low': 0.0 ,'Low': 1.0, 'Medium': 2.0, 'Very high': 3.0, 'High': 4.0})
test_data[['F12', 'F12_encoded']]

,F12,F12_encoded
0,Low,1.0
1,Medium,2.0
2,Low,1.0
3,High,4.0
4,Low,1.0
...,...,...
1495,Very high,3.0
1496,Very low,0.0
1497,Low,1.0
1498,Medium,2.0


In [ ]:
#spliting values in column F4 into other columns
#using one-hot encoding in F4
f4_encoded = pd.get_dummies(test_data['F4'], prefix='F4')
f4_encoded


,F4_Europe,F4_Rest,F4_UK,F4_USA
0,0,0,0,1
1,0,1,0,0
2,0,1,0,0
3,0,0,1,0
4,0,1,0,0
...,...,...,...,...
1495,0,1,0,0
1496,0,1,0,0
1497,1,0,0,0
1498,0,0,1,0


In [ ]:
#adding newly converted data back into data frame
test_data = pd.concat([f4_encoded,test_data], axis = 1 )
test_data

,F4_Europe,F4_Rest,F4_UK,F4_USA,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F12_encoded
0,0,0,0,1,28.70,0.82,-115.06,USA,-280.95,581.10,0.06,214.35,2,2,-0.88,Low,-14794.01,-15.09,-35.61,1578.68,1.0
1,0,1,0,0,5.40,6.90,-170.24,Rest,-228.00,330.54,0.31,-164.98,1,9,0.28,Medium,-11632.88,0.72,-29.73,1647.02,2.0
2,0,1,0,0,21.96,4.40,-219.12,Rest,-59.67,662.28,4.98,281.54,6,5,5.82,Low,-12631.29,-23.61,-32.73,1449.14,1.0
3,0,0,1,0,10.52,1.02,-184.14,UK,-211.23,506.55,0.57,-311.96,7,5,1.92,High,-26664.88,1.80,-22.92,911.98,4.0
4,0,1,0,0,2.00,0.58,-190.56,Rest,-284.82,649.41,0.05,291.47,8,3,-0.02,Low,-9901.41,-13.47,-48.81,2251.04,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0,1,0,0,18.42,8.22,-213.52,Rest,-189.33,569.43,0.09,-315.11,2,3,8.64,Very high,-16759.93,-13.56,-37.08,869.80,3.0
1496,0,1,0,0,12.06,14.46,-251.02,Rest,-87.90,640.32,3.67,201.37,7,8,9.34,Very low,-11645.46,-4.35,-56.10,2040.40,0.0
1497,1,0,0,0,6.54,10.42,-156.28,Europe,-221.82,462.30,0.00,-267.50,2,6,5.00,Low,-23752.87,-8.37,-21.27,1130.64,1.0
1498,0,0,1,0,8.06,13.34,-211.82,UK,-483.24,766.38,0.19,140.38,4,6,5.34,Medium,-13748.25,-9.21,-18.96,1261.38,2.0


In [ ]:
test_data = test_data.drop(["F4", "F12"], axis = 1)


In [ ]:
test_data

,F4_Europe,F4_Rest,F4_UK,F4_USA,F1,F2,F3,F5,F6,F7,F8,F9,F10,F11,F13,F14,F15,F16,F12_encoded
0,0,0,0,1,28.70,0.82,-115.06,-280.95,581.10,0.06,214.35,2,2,-0.88,-14794.01,-15.09,-35.61,1578.68,1.0
1,0,1,0,0,5.40,6.90,-170.24,-228.00,330.54,0.31,-164.98,1,9,0.28,-11632.88,0.72,-29.73,1647.02,2.0
2,0,1,0,0,21.96,4.40,-219.12,-59.67,662.28,4.98,281.54,6,5,5.82,-12631.29,-23.61,-32.73,1449.14,1.0
3,0,0,1,0,10.52,1.02,-184.14,-211.23,506.55,0.57,-311.96,7,5,1.92,-26664.88,1.80,-22.92,911.98,4.0
4,0,1,0,0,2.00,0.58,-190.56,-284.82,649.41,0.05,291.47,8,3,-0.02,-9901.41,-13.47,-48.81,2251.04,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0,1,0,0,18.42,8.22,-213.52,-189.33,569.43,0.09,-315.11,2,3,8.64,-16759.93,-13.56,-37.08,869.80,3.0
1496,0,1,0,0,12.06,14.46,-251.02,-87.90,640.32,3.67,201.37,7,8,9.34,-11645.46,-4.35,-56.10,2040.40,0.0
1497,1,0,0,0,6.54,10.42,-156.28,-221.82,462.30,0.00,-267.50,2,6,5.00,-23752.87,-8.37,-21.27,1130.64,1.0
1498,0,0,1,0,8.06,13.34,-211.82,-483.24,766.38,0.19,140.38,4,6,5.34,-13748.25,-9.21,-18.96,1261.38,2.0


In [ ]:
# predict target variables on the test data
predictions = best_model.predict(test_data)

In [ ]:
pred_df = pd.DataFrame(predictions)

In [ ]:
pred_df.value_counts()

 4235.295267    1
 462.540538     1
 447.919307     1
 448.570893     1
 449.810966     1
               ..
 1262.459632    1
 1263.013739    1
 1266.805918    1
 1268.454728    1
-2034.650441    1
Length: 1500, dtype: int64

In [ ]:
# Save the predictions to a test_data 
test_data['Target'] = predictions



In [ ]:
test_data.head()

,F4_Europe,F4_Rest,F4_UK,F4_USA,F1,F2,F3,F5,F6,F7,F8,F9,F10,F11,F13,F14,F15,F16,F12_encoded,Target
0,0,0,0,1,28.70,0.82,-115.06,-280.95,581.10,0.06,214.35,2,2,-0.88,-14794.01,-15.09,-35.61,1578.68,1.0,737.936466
1,0,1,0,0,5.40,6.90,-170.24,-228.00,330.54,0.31,-164.98,1,9,0.28,-11632.88,0.72,-29.73,1647.02,2.0,283.776919
2,0,1,0,0,21.96,4.40,-219.12,-59.67,662.28,4.98,281.54,6,5,5.82,-12631.29,-23.61,-32.73,1449.14,1.0,2615.000077
3,0,0,1,0,10.52,1.02,-184.14,-211.23,506.55,0.57,-311.96,7,5,1.92,-26664.88,1.80,-22.92,911.98,4.0,1934.233762
4,0,1,0,0,2.00,0.58,-190.56,-284.82,649.41,0.05,291.47,8,3,-0.02,-9901.41,-13.47,-48.81,2251.04,1.0,123.420848


In [ ]:
test_data.to_csv(r'/content/drive/MyDrive/Design and application of a machine learning system for a practical problem/final submission/CE802_P3_Test.csv', index= False)